In [23]:
import string, os 
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import pandas as pd
import numpy as np
import string, os 
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [24]:
import pandas as pd

# Import tweet data

In [61]:
finaldf = pd.read_csv("/Users/maharshi/Desktop/Fall 2019/STOR 565/project/Tweets/tweets-20191106-1.csv")

In [62]:
from sklearn.preprocessing import LabelEncoder
encod = LabelEncoder()
finaldf['partyNum'] = encod.fit_transform(finaldf['party'].astype(str))

In [63]:
finaldf.head()

,TweetId,CleanText,party,partyNum
0,1190407200109318144,whistleblower must come forward explain accoun...,republican,1
1,1190400372180807680,impeach someone done anything wrong,republican,1
2,1190385966621569024,oh beto dropped race president despite saying ...,republican,1
3,1190383074632257536,republicans never unified right dems mess corr...,republican,1
4,1190375834529468416,president federal government wants wonerful ci...,republican,1


# train-test split

In [28]:
from sklearn.model_selection import train_test_split


In [64]:
X_train,X_test,y_train,y_test = train_test_split(finaldf,list(finaldf['partyNum'].values),test_size=0.2)

In [65]:
print(X_train.shape,X_test.shape)
print(len(finaldf))

(1267, 4) (317, 4)
1584


# Use Pre-trained Glove Embeddings:

In [66]:
vocabulary_size = 30000
tokenizer = Tokenizer(num_words= vocabulary_size,lower=False,split=' ')
tokenizer.fit_on_texts((X_train['CleanText'].values))

sequences = tokenizer.texts_to_sequences((X_train['CleanText'].values))
data = pad_sequences(sequences,maxlen=50)

print(data.shape)

(1267, 50)


In [67]:
len((X_train['CleanText'].values))

1267

In [68]:
embeddings_index = dict()
f = open('/Users/maharshi/Desktop/Fall 2019/STOR 565/project/glove.twitter.27B/glove.twitter.27B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 1193514 word vectors.


In [69]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [70]:
len(embedding_matrix)

30000

# Network Architecture:

In [71]:
mod = Sequential()
mod.add(Embedding(vocabulary_size, 100, input_length=50, weights=[embedding_matrix], trainable=False))
mod.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
mod.add(Dense(1, activation='sigmoid'))
mod.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [72]:
mod.fit(data, np.array(y_train), validation_split=0.4, epochs = 20)

Train on 760 samples, validate on 507 samples
Epoch 1/20
760/760 [==============================] - 3s 4ms/step - loss: 0.6135 - accuracy: 0.6803 - val_loss: 0.5118 - val_accuracy: 0.7890
Epoch 2/20
760/760 [==============================] - 2s 3ms/step - loss: 0.4483 - accuracy: 0.8026 - val_loss: 0.4459 - val_accuracy: 0.8028
Epoch 3/20
760/760 [==============================] - 2s 3ms/step - loss: 0.3985 - accuracy: 0.8276 - val_loss: 0.4231 - val_accuracy: 0.8343
Epoch 4/20
760/760 [==============================] - 2s 3ms/step - loss: 0.3704 - accuracy: 0.8276 - val_loss: 0.4029 - val_accuracy: 0.8363
Epoch 5/20
760/760 [==============================] - 2s 3ms/step - loss: 0.3186 - accuracy: 0.8684 - val_loss: 0.4342 - val_accuracy: 0.8284
Epoch 6/20
760/760 [==============================] - 2s 3ms/step - loss: 0.3313 - accuracy: 0.8553 - val_loss: 0.3837 - val_accuracy: 0.8402
Epoch 7/20
760/760 [==============================] - 2s 3ms/step - loss: 0.3043 - accuracy: 0.8724 - 

In [112]:
new_complaint = ['stock market up big enjoy','equal']
seq1 = tokenizer.texts_to_sequences(new_complaint)
da1 = pad_sequences(seq1, maxlen=50)
pred1 = mod.predict(da1)
#labels=['Republican','Democrat']
#print(pred1, labels[np.argmax(pred1)])
pred1

array([[0.6589487 ],
       [0.23531099]], dtype=float32)

0

In [54]:
seq

[[]]

In [55]:
mod.predict(padded)

array([[0.5986774]], dtype=float32)

In [34]:
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

TENSORBOARD_FILES_PATH = "/Users/maharshi/Desktop/Fall 2019/STOR 565/project/TensorBoard"

ModuleNotFoundError: No module named 'tensorflow.contrib'

In [42]:
mod.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 100)           10000000  
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 10,080,501
Trainable params: 80,501
Non-trainable params: 10,000,000
_________________________________________________________________


In [76]:
data[3]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       2195,  428,   29,   30,  387,  120,  292,  121,  122,  655, 1481,
        556,   93, 2196,  187,  656,   57, 1126,   50,  100,  429,    2,
        247, 2197,  268,  317,   19,   34], dtype=int32)

In [77]:
sequences[3]

[2195,
 428,
 29,
 30,
 387,
 120,
 292,
 121,
 122,
 655,
 1481,
 556,
 93,
 2196,
 187,
 656,
 57,
 1126,
 50,
 100,
 429,
 2,
 247,
 2197,
 268,
 317,
 19,
 34]